<a href="https://colab.research.google.com/github/flohaha/Colab_InsurTeam_Nobis/blob/main/nobis_genai_langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### https://medium.com/@amangrulkar18/build-a-docbot-implementing-rag-with-langchain-chroma-and-llm-d38502875003

In [1]:
! pip install langchain langchain-community langchain-openai chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.3 MB/s eta 0:00:00


In [2]:

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter

In [ ]:
link_contract = 'https://drive.google.com/file/d/1eJaGSWbqy2Yp-Y4k6TrKaKUoHzTpN5K6/view'

In [12]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1eJaGSWbqy2Yp-Y4k6TrKaKUoHzTpN5K6',
                                    dest_path='./nobis_contract.pdf',
                                    unzip=True)

In [15]:
 from google.colab import drive
 drive.mount('/content/drive')
 !ls -l /content/drive/'My Drive/PRO/InsurTech-SysTech/SysTech_AI/Nobis_Cancellation_Rules'

Mounted at /content/drive
total 462564
drwx------ 2 root root      4096 Apr  5 11:38  Archive
-rw------- 1 root root       172 Jan 26 13:52 'BRE - Solutions Comparison.gsheet'
-rw------- 1 root root      9446 Jun 11 11:39  GoRule_Nobis_Example.ipynb
-rw------- 1 root root  33069845 Apr  5 11:16 'InsurTeam_Nobis_Demo_l ‐ Réalisée avec Clipchamp.mp4'
-rw------- 1 root root  21057305 Apr  5 11:26 'InsurTeam_Nobis_Demo_ml ‐ Réalisée avec Clipchamp.mp4'
-rw------- 1 root root  13634656 Apr  5 11:46  InsurTeam_Nobis_Demo_s.mp4
-rw------- 1 root root    160768 Jun 15 09:50 'Nobis - Cancellation EN 12.23.doc'
-rw------- 1 root root    643298 Jun 15 09:54 'Nobis - Cancellation EN 12.23.pdf'
-rw------- 1 root root 405053846 Feb  9 17:53  Python_GoRules_Zen_Nobis_Example_Video.mp4
-rw------- 1 root root     23293 Jun 11 07:47  Rule_Engine_Gorules_Editor_Nobis_FINAL10.json
-rw------- 1 root root      5347 Apr  5 08:15  table_nobis_cancel.csv
-rw------- 1 root root       172 Apr  5 10:19  table

In [17]:
contract_file = '/content/drive/My Drive/PRO/InsurTech-SysTech/SysTech_AI/Nobis_Cancellation_Rules/Nobis - Cancellation EN 12.23.pdf'

In [18]:
import pypdf

# load the document pdf
loader_pdf = PyPDFLoader(contract_file)
pages_pdf = loader_pdf.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(pages_pdf)

In [21]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('Colab_OpenAi_Key_Insurteam')

In [22]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding_function = OpenAIEmbeddings()

# load docs into Chroma
vector_db = Chroma.from_documents(docs, embedding_function, persist_directory='persist_directory_path')

# Helpful to force a save
vector_db.persist()

# get db connection
vector_db_connection = Chroma(persist_directory='persist_directory_path', embedding_function=embedding_function)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [23]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter

# create a retriever
retriever = vector_db_connection.as_retriever(search_kwargs={"k": 3})

# create embeddings
embeddings = OpenAIEmbeddings()

# create embeddings filter
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)

# create a compression retriever filter using retriever and embeddings
compression_retriever_filter = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

In [24]:
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# create a retrieval qa chain using llm
llm = ChatOpenAI(temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever_filter,
                                 verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
# Ask Question

question = '''what are the conditions to get reimbursed for cancellation ?
answer in bullet points in simple terms and add reference to which article of the contract it is to illustrate your points'''

answer = qa(question)
print(answer['result'])

##### RESPONSE #####
# > Entering new RetrievalQA chain...
#
# > Finished chain.
# {'query': 'Who is the instructor for this course?',
#  'result': 'The instructor for this course on Kubernetes for Absolute Beginners is Mumshad Mannambeth.'}



> Entering new RetrievalQA chain...

> Finished chain.
- The cancellation must be due to unforeseeable circumstances at the time of booking the trip (Article 1.1).
- The cancellation must be a result of the death, illness, or injury requiring hospitalization of the insured, their family members up to the 3rd degree of kinship, or other specific individuals listed in the contract (Article 1.1).
- The insured must report the cancellation by midnight of the day following the event either by phone or online (Article 6.4).
- The insured must inform the tour operator or travel agency where the booking was made about the cancellation (Article 6.4).
- The insured must cooperate with the company's investigations and assessments, providing all necessary documentation and allowing access to medical professionals involved in their case (Article 6.4).


In [38]:
# Ask Question

question = '''what is the amount i can get reimbursed for cancellation
answer in bullet points in simple terms and add reference to which article of the contract it is to illustrate your points
'''

answer = qa(question)
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.
- The maximum amount per insured for reimbursement in case of cancellation is €5,000.00 (Art. 1.2).
- The limit per event, affecting one or more persons connected by the purchase of the same trip, is €15,000.00 (Art. 1.2).
- The total cost of the trip is covered within the maximum amount per insured (Art. 1.2).
- The reimbursement will be made without the application of any excess (Art. 1.2).
- The compensation payable to the insured is equal to the withdrawal fee calculated as of the date of the event (Art. 6.4).


In [39]:
# Ask Question

question = '''what documents should i provide to get reimbursed for cancellation
answer in bullet points in simple terms and add reference to which article of the contract it is to illustrate your points
'''

answer = qa(question)
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.
- You need to provide a report of the cancellation by contacting the toll-free number 800.894124 or the 24-hour number 039/9890.703, or online at www.nobis.it - section "On-Line Reporting" (Article 6.4).
- Notify the Tour Operator or Travel Agency where you booked the trip about the cancellation (Article 6.4).
- Allow the insurance company to conduct necessary investigations and assessments by providing all documentation related to the cancellation (Article 6.4).
- Release any doctors who have treated you from professional secrecy if needed for the claim evaluation (Article 6.4).


In [40]:
# Ask Question

question = '''what are the precise documents should i provide to get reimbursed for cancellation
answer in bullet points in simple terms and add reference to which article of the contract it is to illustrate your points
'''

answer = qa(question)
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.
- You need to provide a report of the event causing the cancellation by contacting the toll-free number 800.894124 or the 24-hour number 039/9890.703, or report online at www.nobis.it - section "On-Line Reporting" (Article 6.4).
- Notify the tour operator or travel agency where you booked the trip about the cancellation (Article 6.4).
- Allow the insurance company to conduct necessary investigations and assessments by providing all documentation related to the cancellation (Article 6.4).
- Release any doctors who have examined or treated you from professional secrecy if needed for the claim evaluation (Article 6.4).


In [42]:
# Ask Question

question = '''when should i report my cancellation in order to get reimbursed - what maximum days after the cancellation?
answer in bullet points in simple terms and add reference to which article of the contract it is to illustrate your points
'''

answer = qa(question)
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.
- You should report your cancellation by midnight of the day following the event that led to the cancellation of the trip. (ART 6.4 – REIMBURSEMENT CRITERIA)
- Failure to report the cancellation promptly may result in the total or partial loss of the right to compensation. (ART 6.4 – REIMBURSEMENT CRITERIA)


In [47]:
# Ask Question

question = '''
provide all the conditions to get reimbursed or not from the nobis cancellation in bullet points
for each bullet point provide the conditions and their output
based on that provide a business rule in a table format
'''

answer = qa(question)
print(answer['result'])



> Entering new RetrievalQA chain...

> Finished chain.
Conditions for Reimbursement from Nobis Travel Cancellation Insurance:

1. **Death, Illness, or Injury Resulting in Hospitalization:**
   - Condition: Death, illness, or injury leading to hospitalization of the Insured or Travel Companion's close family members.
   - Output: Eligible for reimbursement.

2. **Covid-19 Infection Requiring Hospitalization:**
   - Condition: Covid-19 infection necessitating hospitalization with proof of vaccination or documented health reasons preventing vaccination.
   - Output: Eligible for reimbursement.

3. **Immediate Reporting Requirement:**
   - Condition: Obligation to report the cancellation by midnight of the day following the event.
   - Output: Failure to report may result in ineligibility for reimbursement.

4. **Cooperation with Investigations:**
   - Condition: Must cooperate with investigations and provide all necessary documentation.
   - Output: Failure to cooperate may result in in